In [1]:
%display latex

In [2]:
N = 3
F = GF(2^3, 'x')
x = F.gen()

# N = 2
# F = GF(2^N, 'x')
# x = F.gen()

In [3]:
basis = [x^3, x^5, x^6] # self-dual basis
# basis = [x, x+1]

In [4]:
# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

In [5]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [6]:
# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [7]:
# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

In [8]:
def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

Verification of the recurrence relation for the straight lines:

In [9]:
for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [-1,-1,-1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


Now for the curve $f$:

In [10]:
for mu in F:
    curve = lambda t: mu * t + x^3 * t^2 + x^5 * t^4
    sols = sign_perm(solve_basis(basis, curve), [1,-1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


So the calculation of the coefficients seem to hold fine. Now let's try to calculate this discrete phase space structure.

In [11]:
def ray(mu):
    return lambda t: mu * t

In [12]:
def f(phi0): 
    return lambda t: phi0 * t + x^3 * t^2 + x^5 * t^4

The point $(\alpha,\mu \alpha)$ corresponds to coefficient $c_{\alpha,\mu}$, so we can loop throught the curve parameters $\mu$, calculate the coefficients and assign them to the point $(\alpha, f_\mu(\alpha))$ in phase space.

In [14]:
def toInt(k):
    return list(F).index(k)

In [15]:
def all_perms():
    perms = []
    for i in [1,-1]:
        for j in [1,-1]:
            for k in [1,-1]:
                perms.append([i,j,k])
    return perms

In [16]:
def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [75]:
PS(ray, [[1,-1,1]] * 8) # all positive solutions

[ 1  1  1  1  1  1  1  1]
[-1  1  I -1  I -I -I  1]
[-1  I  1 -I  I -I -1  1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[-1 -I -I -1 -1 -I -1 -I]
[ 1  I  1  1  I  1  I  I]
[-1  1  1 -I  1 -I -I  I]

In [78]:
PS(f, [[-1,-1,-1]] * 8) 

[ 1  1  1  1  1  1  1  1]
[ 1 -1  I  1  I -I -I -1]
[ 1  I -1 -I  I -I  1 -1]
[-1 -1 -I -I -I -1 -1 -I]
[ 1  I  I -I -1  1 -I -1]
[-1 -I -I -1 -1 -I -1 -I]
[-1 -I -1 -1 -I -1 -I -I]
[-1 -1  1  I -1  I -I  I]

We have 8 possible sign permutations for each of the 8 curve parameters, so we have $8^8$ possible phase space structures! Of course there is a high number of repeated structures given that the multiplicative term in the general coefficient function can only actually take on 4 possible values. We ignore these details for practicality.

In [30]:
# Rays plane
# Curve plane
m = zero_matrix(SR, 8, 8)
for j, mu in enumerate(F):
    for i, a in enumerate(F):
        b = ray(mu)(a)
        m[i, toInt(b)] = j+1
m

[8 0 0 0 0 0 0 0]
[1 8 2 3 4 5 6 7]
[1 7 8 2 3 4 5 6]
[1 6 7 8 2 3 4 5]
[1 5 6 7 8 2 3 4]
[1 4 5 6 7 8 2 3]
[1 3 4 5 6 7 8 2]
[1 2 3 4 5 6 7 8]

In [31]:
list(F)[1]

x

In [32]:
c(x, ray(x))

-I

In [29]:
# Curve plane
m = zero_matrix(SR, 8, 8)
for j, mu in enumerate(F):
    for i, a in enumerate(F):
        b = f(mu)(a)
        m[i, toInt(b)] = j+1
m

[8 0 0 0 0 0 0 0]
[3 7 5 1 6 2 4 8]
[8 3 1 4 7 2 6 5]
[3 4 8 7 5 1 6 2]
[2 3 7 6 4 1 5 8]
[6 3 8 1 2 5 7 4]
[1 3 4 5 6 7 8 2]
[3 5 1 6 2 4 8 7]

In [24]:
list(F)[4]

x^2 + x

In [25]:
c(x, f(x^2+x))

I

In [36]:
ray(x)(x) == f(x^2+x)(x)

True

In [37]:
f(x^2+x)(x)

x^2

In [56]:
from itertools import product

In [69]:
len(list(product(all_perms(), repeat=8)))

16777216

In [74]:
count = 0
for perm1 in product(all_perms(), repeat=8):
    for perm2 in product(all_perms(), repeat=8):
        if PS(ray, perm1) == PS(f, perm2):
            print('Match!', perm1, perm2)
            break
        count += 1
        if count % 100 == 0:
            print('Computed {} curve coefficient matrices'.format(count))

Computed 100 ray coefficient matrices
Computed 200 ray coefficient matrices
Computed 300 ray coefficient matrices
Computed 400 ray coefficient matrices
Computed 500 ray coefficient matrices
Computed 600 ray coefficient matrices
Computed 700 ray coefficient matrices
Computed 800 ray coefficient matrices
Computed 900 ray coefficient matrices
Computed 1000 ray coefficient matrices
Computed 1100 ray coefficient matrices
Computed 1200 ray coefficient matrices
Computed 1300 ray coefficient matrices
Computed 1400 ray coefficient matrices
Computed 1500 ray coefficient matrices
Computed 1600 ray coefficient matrices


KeyboardInterrupt: 